# Importing libraries

In [1]:
import pandas as pd
import numpy as np

from sklearn.decomposition import PCA

from category_encoders import TargetEncoder

from imblearn.over_sampling import SMOTE

from xgboost import XGBClassifier
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score, fbeta_score, accuracy_score, roc_auc_score

# Reading file and tidying

In [7]:
df_train = pd.read_csv("../dataset/train.csv")
df_test = pd.read_csv("../dataset/test.csv")

,income,age,experience,marital_status,car_ownership,profession,city,state,rented,risk_flag
0,0.327124,0.603448,1.0,1,0,Technology_specialist,Gorakhpur,Uttar_Pradesh,1.0,0
1,0.130675,0.551724,0.2,1,0,Civil_engineer,Thoothukudi,Tamil_Nadu,1.0,0
2,0.501596,0.448276,1.0,1,0,Magistrate,Khammam,Telangana,1.0,0
3,0.281679,0.568966,0.0,1,0,Chef,Serampore,West_Bengal,1.0,0
4,0.878926,0.344828,0.1,1,0,Designer,Tiruppur,Tamil_Nadu,1.0,0


In [3]:
y_train = df_train["risk_flag"]
x_train = df_train.drop("risk_flag", axis=1)

y_test = df_test["risk_flag"]
x_test = df_test.drop("risk_flag", axis=1)

In [4]:
def target_encoding(df_x, df_y):
    x = df_x.copy()
    
    # Target Encoding — categorical columns with high cardinality: profession, city, state
    profession_target_enc = TargetEncoder()
    x["profession_encoded"] = profession_target_enc.fit_transform(x["profession"], df_y)
    
    city_target_enc = TargetEncoder()
    x["city_encoded"] = city_target_enc.fit_transform(x["city"], df_y)
    
    state_target_enc = TargetEncoder()
    x["state_encoded"] = state_target_enc.fit_transform(x["state"], df_y)
    
    x.drop("profession", axis=1, inplace=True)
    x.drop("city", axis=1, inplace=True)
    x.drop("state", axis=1, inplace=True)
    return x

x_train = target_encoding(x_train, y_train)
x_test = target_encoding(x_test, y_test)

/Users/jonw/miniforge3/envs/ml_env/lib/python3.8/site-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


In [5]:
# # https://xgboost.readthedocs.io/en/latest/python/python_api.html#module-xgboost.sklearn
xgb = XGBClassifier(use_label_encoder=False, random_state=2021)
      
oversampler = SMOTE(random_state=2021)
x_train, y_train = oversampler.fit_resample(x_train, y_train)

xgb.fit(x_train, y_train, eval_metric='logloss')
y_pred = xgb.predict(x_test)

print(f"-----------------------TEST SCORES-----------------------")
print(f"Recall: {recall_score(y_test, y_pred)}")
print(f"Precision: {precision_score(y_test, y_pred)}")
print(f"F2-Score: {fbeta_score(y_test, y_pred, beta=2)}")
print(f"Accuracy score: {accuracy_score(y_test, y_pred)}")
print(f"AUC Score: {roc_auc_score(y_test, y_pred)}")
print()

-----------------------TEST SCORES-----------------------
Recall: 0.9537022100338765
Precision: 0.13576136128780397
F2-Score: 0.4325241795063136
Accuracy score: 0.24757936507936507
AUC Score: 0.551125442701606



In [6]:
# model = XGBClassifier(n_jobs=-1, random_state=0)

# param_dist = {
#     "learning_rate": [0.05, 0.10, 0.15, 0.20, 0.25, 0.30],
#     "max_depth": [3, 4, 5, 6, 8, 10, 12, 15],
#     'min_child_weight': [1, 3, 5, 7],
#     'gamma': [0.0, 0.1, 0.2 , 0.3, 0.4],
#     'colsample_bytree': [0.3, 0.4, 0.5 , 0.7],
# }

# grid_search = GridSearchCV(model, param_dist, n_jobs=-1, scoring="recall", error_score='raise')

# grid_search.fit(x_train, y_train)
# grid_search.best_params_